In [1]:
import pandas as pd
import numpy as np
import requests
import re
from math import radians, sin, cos, sqrt, atan2
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


df = pd.read_csv(r"C:\Users\sevva\OneDrive\Masaüstü\atık1amk\data\guncellenmis_waste_detect (1).csv")

kriterler = [
    "fiyat", "sistem_suresi", "toplam_satis", "puanlama",
    "geri_donusum_lisansi", "karbon_ayak_izi",
    "lojistik_uygunluk", "eslesme_orani", "mesafe_km"
]


weights = np.array([0.05, 0.07, 0.03, 0.15, 0.05, 0.1, 0.15, 0.2, 0.2])  # Örnek ağırlıklar

scaler = MinMaxScaler()
normalized = scaler.fit_transform(df[kriterler])
weighted = normalized * weights
pis = weighted.max(axis=0)
nis = weighted.min(axis=0)
d_pis = np.linalg.norm(weighted - pis, axis=1)
d_nis = np.linalg.norm(weighted - nis, axis=1)
df["C_skoru"] = d_nis / (d_nis + d_pis)
sorted_df = df.sort_values("C_skoru", ascending=False).reset_index(drop=True)


X = sorted_df[kriterler]
y = sorted_df["firma_adi"]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
model = XGBClassifier(objective="multi:softmax", use_label_encoder=False, eval_metric="mlogloss")
model.fit(X_train, y_train)

api_key = "5b3ce3597851110001cf6248d32a9ee6e3d94103ad5867f69dc7b74a"

def adres_to_coords(adres, api_key):
    try:
        url = "https://api.openrouteservice.org/geocode/search"
        params = {"api_key": api_key, "text": adres, "boundary.country": "TR"}
        r = requests.get(url, params=params)
        data = r.json()
        coords = data["features"][0]["geometry"]["coordinates"]
        return coords[::-1]
    except:
        return None

from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Dünya'nın yarıçapı (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return round(R * c, 2)

    



def firma_tahmini_yap(adres, api_key, model, df, features, label_encoder, atik_kodu):
    df = df[df["waste"] == atik_kodu].copy()
    if df.empty:
        return "❌ Bu atık koduna uygun firma bulunamadı."
    k_coords = adres_to_coords(adres, api_key)
    if not k_coords:
        return "❌ Koordinat bulunamadı."
    df["mesafe_km"] = df.apply(lambda row: haversine(k_coords[0], k_coords[1], row["lat"], row["lon"]), axis=1)
    X_input = df[features].mean().to_frame().T
    pred_encoded = model.predict(X_input)
    pred_label = label_encoder.inverse_transform(pred_encoded)[0]
    return pred_label

def onerilen_firma_raporu(firma_adi, veri_df):
    secilen = veri_df[veri_df["firma_adi"] == firma_adi].sort_values("C_skoru", ascending=False).iloc[0]
    return f"""
🔍 En Uygun Alıcı Firma Önerisi (Atık Kodu: {secilen['waste']}):

🏢 Firma: {secilen['firma_adi']}
📍 Adres: {secilen['adres']}
📊 TOPSIS Skoru: {secilen['C_skoru']:.3f}

🎯 Gerekçeler:
✅ Kısa Mesafe → {secilen['mesafe_km']} km
✅ Düşük Fiyat → {secilen['fiyat']} ₺
✅ Karbon Ayak İzi → {secilen['karbon_ayak_izi']}
✅ Yüksek Puanlama → {secilen['puanlama']} / 5
✅ Yüksek Atık Uyumluluğu → %{secilen['eslesme_orani']}
✅ Lojistik Uygunluk → {secilen['lojistik_uygunluk']}
✅ Lisanslı Geri Dönüşüm Firması → {'✅' if secilen['geri_donusum_lisansi'] == 1 else '❌'}

📌 Bu firma, çevresel sürdürülebilirlik ve ekonomik uygunluk açısından en iyi eşleşmeyi sunmaktadır.
"""


adres = input("Adresinizi girin (örn: Ostim OSB, 1243. Sk. No:3, Yenimahalle, Ankara): ")
atik_kodu = input("Atık kodunu girin (örn: 01 04 09): ")

firma = firma_tahmini_yap(adres, api_key, model, df, kriterler, label_encoder, atik_kodu)
print(onerilen_firma_raporu(firma, df))

D:\Python\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:47:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Adresinizi girin (örn: Ostim OSB, 1243. Sk. No:3, Yenimahalle, Ankara):  Ostim OSB, 1243. Sk. No:3, Yenimahalle, Ankara
Atık kodunu girin (örn: 01 04 09):  01 04 09



🔍 En Uygun Alıcı Firma Önerisi (Atık Kodu: 16 05 04*):

🏢 Firma: Temiz Gelecek A.Ş.
📍 Adres: İVEDİK OSB
📊 TOPSIS Skoru: 0.772

🎯 Gerekçeler:
✅ Kısa Mesafe → 379.69 km
✅ Düşük Fiyat → 43 ₺
✅ Karbon Ayak İzi → 1.72
✅ Yüksek Puanlama → 4.89 / 5
✅ Yüksek Atık Uyumluluğu → %87.63
✅ Lojistik Uygunluk → 7
✅ Lisanslı Geri Dönüşüm Firması → ✅

📌 Bu firma, çevresel sürdürülebilirlik ve ekonomik uygunluk açısından en iyi eşleşmeyi sunmaktadır.

